# Extensive CNN Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import tensorflow as tf
from tensorflow.keras import layers, models

AUTOTUNE = tf.data.AUTOTUNE

DATA_PATH = "/content/drive/MyDrive/BrainScanning/CombinedDataset/train"
SAVE_PATH = "/content/drive/MyDrive/BrainScanning/results/extensive_cnn_model.h5"

IMG_SIZE = 150
BATCH = 32

print("TensorFlow:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
!nvidia-smi


def list_image_files(root):
    classes = sorted(
        [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
    )

    paths, labels = [], []
    for idx, cls in enumerate(classes):
        folder = os.path.join(root, cls)
        for f in os.listdir(folder):
            if f.lower().endswith(("png", "jpg", "jpeg")):
                paths.append(os.path.join(folder, f))
                labels.append(idx)
    return paths, labels, classes


def safe_decode(path):
    img_bytes = tf.io.read_file(path)
    img = tf.io.decode_image(
        img_bytes,
        channels=3,
        expand_animations=False,
        dtype=tf.float32,
    )
    shape = tf.shape(img)
    valid = tf.logical_and(shape[0] > 0, shape[1] > 0)
    return img, valid


def process(path, label):
    img, valid = safe_decode(path)
    img = tf.cond(valid, lambda: img, lambda: tf.zeros([1, 1, 3], tf.float32))
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img, label, valid


def build_full_dataset(paths, labels):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(process, num_parallel_calls=AUTOTUNE)
    ds = ds.filter(lambda img, label, valid: valid)
    ds = ds.map(lambda img, label, valid: (img, label), num_parallel_calls=AUTOTUNE)
    return ds


paths, labels, class_names = list_image_files(DATA_PATH)
print("Classes:", class_names)

full_ds = build_full_dataset(paths, labels)

total = sum(1 for _ in full_ds)
print("Total valid images:", total)

full_ds = full_ds.shuffle(total, reshuffle_each_iteration=False)

train_size = int(total * 0.8)
val_size = total - train_size

train_ds = (
    full_ds
    .take(train_size)
    .shuffle(1000)
    .repeat()
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)

val_ds = (
    full_ds
    .skip(train_size)
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)

steps_per_epoch = train_size // BATCH
validation_steps = max(1, val_size // BATCH)


data_aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = data_aug(inputs)

    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inputs, outputs)


num_classes = len(class_names)
model = build_model(num_classes)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=25,
)

os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
model.save(SAVE_PATH)
print("Model saved:", SAVE_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TensorFlow: 2.19.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mon Dec  1 01:57:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 150, 150, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 75, 75, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 37, 37, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │    10,617,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,712,260 (40.86 MB)

 Trainable params: 10,711,812 (40.86 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 102s 283ms/step - accuracy: 0.3544 - loss: 1.8045 - val_accuracy: 0.2661 - val_loss: 2.2382
Epoch 2/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 71s 280ms/step - accuracy: 0.4858 - loss: 1.1070 - val_accuracy: 0.3281 - val_loss: 5.4290
Epoch 3/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 72s 281ms/step - accuracy: 0.5451 - loss: 0.9858 - val_accuracy: 0.3384 - val_loss: 1.3846
Epoch 4/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 70s 276ms/step - accuracy: 0.5660 - loss: 0.9297 - val_accuracy: 0.6758 - val_loss: 0.7045
Epoch 5/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 71s 276ms/step - accuracy: 0.5656 - loss: 0.8963 - val_accuracy: 0.6553 - val_loss: 0.7110
Epoch 6/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 72s 280ms/step - accuracy: 0.5896 - loss: 0.8443 - val_accuracy: 0.6782 - val_loss: 0.6385
Epoch 7/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 68s 268ms/step - accuracy: 0.5982 - loss: 0.8244 - val_accuracy: 0.6626 - val_loss: 0.6579
Epoch 8/25
256/256 ━━━━━━━━━━━━━━━━━━━━ 66s 260ms/step - accuracy: 0.6116 - loss: 

Model saved: /content/drive/MyDrive/BrainScanning/results/extensive_cnn_model.h5
